In [1]:
# imports
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup
import pandas as pd
from scipy import stats
import numpy as np
import spacy
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from urllib.request import urlopen
import json
import dill

In [2]:
# url for scraping 2019 season
url='https://www.hockey-reference.com/leagues/NHL_2019_goalies.html'

In [3]:
# assigning variable to requests getting this url
r=requests.get(url)

In [4]:
# creating soup object
soup = BeautifulSoup(r.content,'html.parser')

In [5]:
#soup.prettify

In [9]:
player_list=[]
for player in soup.find_all('tbody'):
    print(player.find_all('tr')[0])

<tr><th class="right" csk="1" data-stat="ranker" scope="row">1</th><td class="left" csk="Allen,Jake" data-append-csv="allenja01" data-stat="player"><a href="/players/a/allenja01.html">Jake Allen</a></td><td class="right" data-stat="age">28</td><td class="left" data-stat="team_id"><a href="/teams/STL/2019.html">STL</a></td><td class="right" data-stat="games_goalie">46</td><td class="right" data-stat="starts_goalie">45</td><td class="right" data-stat="wins_goalie">19</td><td class="right" data-stat="losses_goalie">17</td><td class="right" data-stat="ties_goalie">8</td><td class="right" data-stat="goals_against">121</td><td class="right" data-stat="shots_against">1277</td><td class="right" data-stat="saves">1156</td><td class="right" data-stat="save_pct">.905</td><td class="right" data-stat="goals_against_avg">2.83</td><td class="right" data-stat="shutouts">3</td><td class="right" data-stat="gps">6.5</td><td class="right" data-stat="min_goalie">2568</td><td class="right" data-stat="qualit

In [10]:
# gathering information on the players I need data from
player_list=[]
for player in soup.find_all('tbody'):
    player_list.append(player.find_all('tr'))

In [11]:
# what does my list look like
player_list

[[<tr><th class="right" csk="1" data-stat="ranker" scope="row">1</th><td class="left" csk="Allen,Jake" data-append-csv="allenja01" data-stat="player"><a href="/players/a/allenja01.html">Jake Allen</a></td><td class="right" data-stat="age">28</td><td class="left" data-stat="team_id"><a href="/teams/STL/2019.html">STL</a></td><td class="right" data-stat="games_goalie">46</td><td class="right" data-stat="starts_goalie">45</td><td class="right" data-stat="wins_goalie">19</td><td class="right" data-stat="losses_goalie">17</td><td class="right" data-stat="ties_goalie">8</td><td class="right" data-stat="goals_against">121</td><td class="right" data-stat="shots_against">1277</td><td class="right" data-stat="saves">1156</td><td class="right" data-stat="save_pct">.905</td><td class="right" data-stat="goals_against_avg">2.83</td><td class="right" data-stat="shutouts">3</td><td class="right" data-stat="gps">6.5</td><td class="right" data-stat="min_goalie">2568</td><td class="right" data-stat="qual

In [12]:
# when I pulled the data it came as a nested list so I need to step inside that list
player_list=player_list[0]

In [13]:
# quick view at what one player looks like
player_list[0]

<tr><th class="right" csk="1" data-stat="ranker" scope="row">1</th><td class="left" csk="Allen,Jake" data-append-csv="allenja01" data-stat="player"><a href="/players/a/allenja01.html">Jake Allen</a></td><td class="right" data-stat="age">28</td><td class="left" data-stat="team_id"><a href="/teams/STL/2019.html">STL</a></td><td class="right" data-stat="games_goalie">46</td><td class="right" data-stat="starts_goalie">45</td><td class="right" data-stat="wins_goalie">19</td><td class="right" data-stat="losses_goalie">17</td><td class="right" data-stat="ties_goalie">8</td><td class="right" data-stat="goals_against">121</td><td class="right" data-stat="shots_against">1277</td><td class="right" data-stat="saves">1156</td><td class="right" data-stat="save_pct">.905</td><td class="right" data-stat="goals_against_avg">2.83</td><td class="right" data-stat="shutouts">3</td><td class="right" data-stat="gps">6.5</td><td class="right" data-stat="min_goalie">2568</td><td class="right" data-stat="qualit

In [14]:
# total amount of players - sort of - there are repeats that will be discussed further later on
len(player_list)

111

In [15]:
# creating list of attributes for first player
a=[]
for stat in player_list[0].find_all('td'):
    a.append(stat.text)

In [17]:
# creating list of attributes for every player
b =[]
for i in range(0,111):
    for stat in player_list[i].find_all('td'):
        b.append(stat.text)

In [18]:
# quick look at beginning of b - what we see is that it captured a little too much and I will need to break this into pieces
b[0:100]

['Jake Allen',
 '28',
 'STL',
 '46',
 '45',
 '19',
 '17',
 '8',
 '121',
 '1277',
 '1156',
 '.905',
 '2.83',
 '3',
 '6.5',
 '2568',
 '25',
 '.556',
 '11',
 '105',
 '-5.90',
 '0',
 '0',
 '0',
 '0',
 'Frederik Andersen',
 '29',
 'TOR',
 '60',
 '60',
 '36',
 '16',
 '7',
 '162',
 '1958',
 '1796',
 '.917',
 '2.77',
 '1',
 '12.5',
 '3510',
 '38',
 '.633',
 '7',
 '92',
 '14.48',
 '0',
 '1',
 '1',
 '2',
 'Craig Anderson',
 '37',
 'OTT',
 '50',
 '47',
 '17',
 '27',
 '4',
 '163',
 '1676',
 '1513',
 '.903',
 '3.51',
 '2',
 '8.1',
 '2785',
 '19',
 '.404',
 '8',
 '108',
 '-11.94',
 '0',
 '2',
 '2',
 '0',
 'Richard Bachman',
 '31',
 'VAN',
 '1',
 '1',
 '0',
 '1',
 '0',
 '6',
 '29',
 '23',
 '.793',
 '6.00',
 '0',
 '-0.2',
 '60',
 '0',
 '.000',
 '1',
 '',
 '',
 '0',
 '0',
 '0',
 '0']

In [20]:
# gauge of how long each player's stats are
b[0:25]

['Jake Allen',
 '28',
 'STL',
 '46',
 '45',
 '19',
 '17',
 '8',
 '121',
 '1277',
 '1156',
 '.905',
 '2.83',
 '3',
 '6.5',
 '2568',
 '25',
 '.556',
 '11',
 '105',
 '-5.90',
 '0',
 '0',
 '0',
 '0']

In [28]:
# list comprehension to split up the list of everything into individual players
chunks = [b[x:x+25] for x in range(0, len(b), 25)]

In [30]:
len(chunks)

107

In [29]:
# what does the first player look like - notice how this is a list within a list at index 0
chunks[0]

['Jake Allen',
 '28',
 'STL',
 '46',
 '45',
 '19',
 '17',
 '8',
 '121',
 '1277',
 '1156',
 '.905',
 '2.83',
 '3',
 '6.5',
 '2568',
 '25',
 '.556',
 '11',
 '105',
 '-5.90',
 '0',
 '0',
 '0',
 '0']

In [31]:
# what does every number correspond to?
for stat in player_list[0].find_all(class_='right'):
    print(stat['data-stat'],stat.text)

ranker 1
age 28
games_goalie 46
starts_goalie 45
wins_goalie 19
losses_goalie 17
ties_goalie 8
goals_against 121
shots_against 1277
saves 1156
save_pct .905
goals_against_avg 2.83
shutouts 3
gps 6.5
min_goalie 2568
quality_starts_goalie 25
quality_start_goalie_pct .556
really_bad_starts_goalie 11
ga_pct_minus 105
gs_above_avg -5.90
goals 0
assists 0
points 0
pen_min 0


In [51]:
# instantiating data frame to hold data
df=pd.DataFrame(columns=['name','age','team','gp','gs','wins','losses','ties','goals_against','shots','saves','save_pct','goals_avg',
                        'shutouts','gps','minutes','quality_starts','quality_start_pct','really_bad_starts','ga_pct_minus',
                        'gs_above_avg','goals_scored','assists','points','plm'],
               index=range(0,107))

In [52]:
# quick look at the empty data frame
df.head()

,name,age,team,gp,gs,wins,losses,ties,goals_against,shots,...,minutes,quality_starts,quality_start_pct,really_bad_starts,ga_pct_minus,gs_above_avg,goals_scored,assists,points,plm
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# features
len(df.columns)

25

In [54]:
# number of players to populate this list
len(chunks[0])

25

In [55]:
# adding everything to the data frame
for i in range(len(df)):
    df.name[i] = chunks[i][0]
    df.age[i]=chunks[i][1]
    df.team[i]=chunks[i][2]
    df.gp[i]=chunks[i][3]
    df.gs[i]=chunks[i][4]
    df.wins[i]=chunks[i][5]
    df.losses[i]=chunks[i][6]
    df.ties[i]=chunks[i][7]
    df.goals_against[i]=chunks[i][8]
    df.shots[i]=chunks[i][9]
    df.saves[i]=chunks[i][10]
    df.save_pct[i]=chunks[i][11]
    df.goals_avg[i]=chunks[i][12]
    df.shutouts[i]=chunks[i][13]
    df.gps[i]=chunks[i][14]
    df.minutes[i]=chunks[i][15]
    df.quality_starts[i]=chunks[i][16]
    df.quality_start_pct[i]=chunks[i][17]
    df.really_bad_starts[i]=chunks[i][18]
    df.ga_pct_minus[i]=chunks[i][19]
    df.gs_above_avg[i]=chunks[i][20]
    df.goals_scored[i]=chunks[i][21]
    df.assists[i]=chunks[i][22]
    df.points[i]=chunks[i][23]
    df.plm[i]=chunks[i][24]

In [56]:
# quick look at data - notice I have repeating name values
df.head()

,name,age,team,gp,gs,wins,losses,ties,goals_against,shots,...,minutes,quality_starts,quality_start_pct,really_bad_starts,ga_pct_minus,gs_above_avg,goals_scored,assists,points,plm
0,Jake Allen,28,STL,46,45,19,17,8,121,1277,...,2568,25,.556,11,105,-5.90,0,0,0,0
1,Frederik Andersen,29,TOR,60,60,36,16,7,162,1958,...,3510,38,.633,7,92,14.48,0,1,1,2
2,Craig Anderson,37,OTT,50,47,17,27,4,163,1676,...,2785,19,.404,8,108,-11.94,0,2,2,0
3,Richard Bachman,31,VAN,1,1,0,1,0,6,29,...,60,0,.000,1,,,0,0,0,0
4,Jonathan Bernier,30,DET,35,30,9,18,5,98,1025,...,1860,13,.433,5,106,-5.61,0,0,0,0


In [ ]:
# tail of data
df.tail()

In [57]:
# data info - notice I will need to change a lot of numbers to be integers or floats
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 25 columns):
name                 107 non-null object
age                  107 non-null object
team                 107 non-null object
gp                   107 non-null object
gs                   107 non-null object
wins                 107 non-null object
losses               107 non-null object
ties                 107 non-null object
goals_against        107 non-null object
shots                107 non-null object
saves                107 non-null object
save_pct             107 non-null object
goals_avg            107 non-null object
shutouts             107 non-null object
gps                  107 non-null object
minutes              107 non-null object
quality_starts       107 non-null object
quality_start_pct    107 non-null object
really_bad_starts    107 non-null object
ga_pct_minus         107 non-null object
gs_above_avg         107 non-null object
goals_scored         107 non-null 

In [58]:
# mapping age as an integer
df.age=df.age.astype(int)

In [59]:
# mapping games played as an integer
df.gp=df.gp.astype(int)

In [66]:
df.gs=df.gs.astype(int)
df.wins=df.wins.astype(int)
df.losses=df.losses.astype(int)
df.goals_against=df.goals_against.astype(int)
df.ties=df.ties.astype(int)
df.losses=df.losses.astype(int)
df.shots=df.shots.astype(int)
df.saves=df.saves.astype(int)
df.save_pct=df.save_pct.astype(float)
df.goals_avg=df.goals_avg.astype(float)
df.losses=df.losses.astype(int)
df.shutouts=df.shutouts.astype(int)
df.minutes=df.minutes.astype(int)
df.gps=df.gps.astype(float)
df.minutes=df.minutes.astype(int)
df.quality_starts=df.quality_starts.astype(int)
df.quality_start_pct=df.quality_starts.astype(float)
df.really_bad_starts=df.really_bad_starts.astype(float)
#df.gs_above_avg=df.gs_above_avg.astype(float)
#df.ga_pct_minus=df.ga_pct_minus.astype(float)
df.goals_scored=df.goals_scored.astype(float)
df.assists=df.assists.astype(int)
df.points=df.points.astype(int)
df.plm=df.plm.astype(int)

In [67]:
# review of data
df.head()

,name,age,team,gp,gs,wins,losses,ties,goals_against,shots,...,minutes,quality_starts,quality_start_pct,really_bad_starts,ga_pct_minus,gs_above_avg,goals_scored,assists,points,plm
0,Jake Allen,28.0,STL,46.0,45,19,17,8,121,1277,...,2568,25,25.0,11.0,105,-5.90,0.0,0,0,0
1,Frederik Andersen,29.0,TOR,60.0,60,36,16,7,162,1958,...,3510,38,38.0,7.0,92,14.48,0.0,1,1,2
2,Craig Anderson,37.0,OTT,50.0,47,17,27,4,163,1676,...,2785,19,19.0,8.0,108,-11.94,0.0,2,2,0
3,Richard Bachman,31.0,VAN,1.0,1,0,1,0,6,29,...,60,0,0.0,1.0,,,0.0,0,0,0
4,Jonathan Bernier,30.0,DET,35.0,30,9,18,5,98,1025,...,1860,13,13.0,5.0,106,-5.61,0.0,0,0,0


In [68]:
# add faceoff percent back as a float
df['mpg']=round((df.minutes)/(df.gp),3)

In [69]:
# adding season column so I can later add more data
df['season']=2019

In [70]:
# review of data
df.head()

,name,age,team,gp,gs,wins,losses,ties,goals_against,shots,...,quality_start_pct,really_bad_starts,ga_pct_minus,gs_above_avg,goals_scored,assists,points,plm,mpg,season
0,Jake Allen,28.0,STL,46.0,45,19,17,8,121,1277,...,25.0,11.0,105,-5.90,0.0,0,0,0,55.826,2019
1,Frederik Andersen,29.0,TOR,60.0,60,36,16,7,162,1958,...,38.0,7.0,92,14.48,0.0,1,1,2,58.500,2019
2,Craig Anderson,37.0,OTT,50.0,47,17,27,4,163,1676,...,19.0,8.0,108,-11.94,0.0,2,2,0,55.700,2019
3,Richard Bachman,31.0,VAN,1.0,1,0,1,0,6,29,...,0.0,1.0,,,0.0,0,0,0,60.000,2019
4,Jonathan Bernier,30.0,DET,35.0,30,9,18,5,98,1025,...,13.0,5.0,106,-5.61,0.0,0,0,0,53.143,2019


In [71]:
#df.gs_above_avg=df.gs_above_avg.astype(float)
#df.ga_pct_minus=df.ga_pct_minus.astype(float)

In [76]:
df.gs_above_avg.replace('',0,inplace=True)

In [77]:
df.gs_above_avg=df.gs_above_avg.astype(float)

In [78]:
df.ga_pct_minus.unique()

array(['105', '92', '108', '', '106', '81', '73', '91', '97', '83', '80',
       '102', '127', '93', '96', '103', '86', '99', '101', '134', '115',
       '120', '114', '104', '78', '112', '98', '95', '90', '126', '116',
       '84', '139', '124', '111', '94', '113', '109', '110', '119'],
      dtype=object)

In [79]:
df.ga_pct_minus.replace('',0,inplace=True)

In [80]:
df.ga_pct_minus=df.ga_pct_minus.astype(float)

In [81]:
# checking names of columns in 2019 season
df.columns

Index(['name', 'age', 'team', 'gp', 'gs', 'wins', 'losses', 'ties',
       'goals_against', 'shots', 'saves', 'save_pct', 'goals_avg', 'shutouts',
       'gps', 'minutes', 'quality_starts', 'quality_start_pct',
       'really_bad_starts', 'ga_pct_minus', 'gs_above_avg', 'goals_scored',
       'assists', 'points', 'plm', 'mpg', 'season'],
      dtype='object')

In [82]:
# figuring out which players changed teams
df[df.name.duplicated()==True].name.unique()

array(['Michael Hutchinson', 'Chad Johnson', 'Mike McKenna',
       'Anders Nilsson', 'Calvin Pickard', 'Anthony Stolarz',
       'Cam Talbot'], dtype=object)

In [83]:
# checking how traded players are represented
df[df.name=='Cam Talbot']

,name,age,team,gp,gs,wins,losses,ties,goals_against,shots,...,quality_start_pct,really_bad_starts,ga_pct_minus,gs_above_avg,goals_scored,assists,points,plm,mpg,season
100,Cam Talbot,31.0,TOT,35.0,32,11,17,3,108,997,...,9.0,10.0,-18.14,-18.14,0.0,1,1,0,54.457,2019
101,Cam Talbot,31.0,EDM,31.0,29,10,15,3,95,888,...,9.0,9.0,-14.96,-14.96,0.0,1,1,0,54.677,2019
102,Cam Talbot,31.0,PHI,4.0,3,1,2,0,13,109,...,0.0,1.0,0.00,0.00,0.0,0,0,0,52.750,2019


In [84]:
# creating data frame with only the final team that player was on - I intend to declare the final team played for as that
# player's team for the season. This way I get a full picture of things like goals and points
final_team_df = df[df.name.duplicated()==True].drop_duplicates(subset='name',keep='last')

In [85]:
# where did "repeat" players end their season
final_team_df.head()

,name,age,team,gp,gs,wins,losses,ties,goals_against,shots,...,quality_start_pct,really_bad_starts,ga_pct_minus,gs_above_avg,goals_scored,assists,points,plm,mpg,season
45,Michael Hutchinson,28.0,FLA,4.0,3,1,1,2,14,87,...,0.0,2.0,0.00,0.00,0.0,0,0,0,50.250,2019
50,Chad Johnson,32.0,ANA,9.0,4,0,5,0,20,156,...,1.0,1.0,0.00,0.00,0.0,0,0,0,35.556,2019
67,Mike McKenna,35.0,PHI,1.0,1,0,1,0,4,24,...,0.0,1.0,0.00,0.00,0.0,0,0,0,57.000,2019
78,Anders Nilsson,28.0,VAN,12.0,12,3,8,1,37,353,...,4.0,2.0,-5.18,-5.18,0.0,0,0,0,59.833,2019
83,Calvin Pickard,26.0,ARI,6.0,4,0,4,0,17,157,...,2.0,0.0,0.00,0.00,0.0,0,0,0,47.167,2019


In [86]:
# reset index of this data frame
final_team_df.reset_index(inplace=True)

In [87]:
# create dictionary to assign values to team name 
final_team_dict = {}
for i in range(len(final_team_df)):
    final_team_dict[final_team_df.name[i]]=final_team_df.team[i]

In [89]:
# quick look at dictionary beginning
list(final_team_dict.items())[0:5]

[('Michael Hutchinson', 'FLA'),
 ('Chad Johnson', 'ANA'),
 ('Mike McKenna', 'PHI'),
 ('Anders Nilsson', 'VAN'),
 ('Calvin Pickard', 'ARI')]

In [90]:
# quick look at dictionary beginning
list(final_team_dict.keys())[0:5]

['Michael Hutchinson',
 'Chad Johnson',
 'Mike McKenna',
 'Anders Nilsson',
 'Calvin Pickard']

In [91]:
# removing repeats of the same player and keeping only total season stats (TOT)
df=df.drop_duplicates(subset='name',keep='first')

In [92]:
# new look of data
df.head()

,name,age,team,gp,gs,wins,losses,ties,goals_against,shots,...,quality_start_pct,really_bad_starts,ga_pct_minus,gs_above_avg,goals_scored,assists,points,plm,mpg,season
0,Jake Allen,28.0,STL,46.0,45,19,17,8,121,1277,...,25.0,11.0,-5.90,-5.90,0.0,0,0,0,55.826,2019
1,Frederik Andersen,29.0,TOR,60.0,60,36,16,7,162,1958,...,38.0,7.0,14.48,14.48,0.0,1,1,2,58.500,2019
2,Craig Anderson,37.0,OTT,50.0,47,17,27,4,163,1676,...,19.0,8.0,-11.94,-11.94,0.0,2,2,0,55.700,2019
3,Richard Bachman,31.0,VAN,1.0,1,0,1,0,6,29,...,0.0,1.0,0.00,0.00,0.0,0,0,0,60.000,2019
4,Jonathan Bernier,30.0,DET,35.0,30,9,18,5,98,1025,...,13.0,5.0,-5.61,-5.61,0.0,0,0,0,53.143,2019


In [93]:
# insuring that name is a string and not object
df.name = df.name.astype(str)

In [95]:
# cheking that my dictionary will map well
'Chad Johnson' in list(final_team_dict.keys())

True

In [96]:
# reset index to deal with players removed
df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)

In [97]:
# replace "TOT" with the final team that player played for
for i in range(len(df)):
    if df.name.iloc[i] in list(final_team_dict.keys()):
        df.team.iloc[i] = final_team_dict[df.name.iloc[i]]

In [98]:
df.head()

,name,age,team,gp,gs,wins,losses,ties,goals_against,shots,...,quality_start_pct,really_bad_starts,ga_pct_minus,gs_above_avg,goals_scored,assists,points,plm,mpg,season
0,Jake Allen,28.0,STL,46.0,45,19,17,8,121,1277,...,25.0,11.0,-5.90,-5.90,0.0,0,0,0,55.826,2019
1,Frederik Andersen,29.0,TOR,60.0,60,36,16,7,162,1958,...,38.0,7.0,14.48,14.48,0.0,1,1,2,58.500,2019
2,Craig Anderson,37.0,OTT,50.0,47,17,27,4,163,1676,...,19.0,8.0,-11.94,-11.94,0.0,2,2,0,55.700,2019
3,Richard Bachman,31.0,VAN,1.0,1,0,1,0,6,29,...,0.0,1.0,0.00,0.00,0.0,0,0,0,60.000,2019
4,Jonathan Bernier,30.0,DET,35.0,30,9,18,5,98,1025,...,13.0,5.0,-5.61,-5.61,0.0,0,0,0,53.143,2019


In [100]:
len(df[df.name.duplicated()==True].name.unique())

0

# Ok... so now I built a scraper that also works for goalies - now I just need to combine this into one function to run through like every different season

In [115]:
def create_goalie_dataframe(year):
    # url to use
    url=f'https://www.hockey-reference.com/leagues/NHL_{year}_goalies.html'
    # accessing via requests
    r=requests.get(url)
    # creating soup object
    soup = BeautifulSoup(r.content,'html.parser')
    # instantiating player list
    player_list=[]
    # looping to populate list
    for player in soup.find_all('tbody'):
        player_list.append(player.find_all('tr'))
    # removing nested element
    player_list=player_list[0]
    # new list
    b =[]
    for i in range(0,len(player_list)):
        for stat in player_list[i].find_all('td'):
            b.append(stat.text)
    # splitting single list into more lists
    chunks = [b[x:x+25] for x in range(0, len(b), 25)]
    # creating data frame to populate
    df=pd.DataFrame(columns=['name','age','team','gp','gs','wins','losses','ties','goals_against','shots','saves','save_pct','goals_avg',
                        'shutouts','gps','minutes','quality_starts','quality_start_pct','really_bad_starts','ga_pct_minus',
                        'gs_above_avg','goals_scored','assists','points','plm'],
               index=range(0,len(chunks)))
    # assigning feature values
    for i in range(len(df)):
        df.name[i] = chunks[i][0]
        df.age[i]=chunks[i][1]
        df.team[i]=chunks[i][2]
        df.gp[i]=chunks[i][3]
        df.gs[i]=chunks[i][4]
        df.wins[i]=chunks[i][5]
        df.losses[i]=chunks[i][6]
        df.ties[i]=chunks[i][7]
        df.goals_against[i]=chunks[i][8]
        df.shots[i]=chunks[i][9]
        df.saves[i]=chunks[i][10]
        df.save_pct[i]=chunks[i][11]
        df.goals_avg[i]=chunks[i][12]
        df.shutouts[i]=chunks[i][13]
        df.gps[i]=chunks[i][14]
        df.minutes[i]=chunks[i][15]
        df.quality_starts[i]=chunks[i][16]
        df.quality_start_pct[i]=chunks[i][17]
        df.really_bad_starts[i]=chunks[i][18]
        df.ga_pct_minus[i]=chunks[i][19]
        df.gs_above_avg[i]=chunks[i][20]
        df.goals_scored[i]=chunks[i][21]
        df.assists[i]=chunks[i][22]
        df.points[i]=chunks[i][23]
        df.plm[i]=chunks[i][24]
    # adding mpg
    df.minutes=df.minutes.astype(float)
    df.gp=df.gp.astype(float)
    df['mpg']=round((df.minutes)/(df.gp),3)
    # assigning season label
    df['season']=int(year)
    # dealing with empty values and assigning float type
    df.gs_above_avg.replace('',0,inplace=True)
    df.gs_above_avg=df.gs_above_avg.astype(float)
    df.ga_pct_minus.replace('',0,inplace=True)
    df.ga_pct_minus=df.ga_pct_minus.astype(float)
    # creating data frame of final team played on in season 
    final_team_df = df[df.name.duplicated()==True].drop_duplicates(subset='name',keep='last')
    final_team_df.reset_index(inplace=True)
    # creating dictionary frame of final team played on in season 
    final_team_dict = {}
    for i in range(len(final_team_df)):
        final_team_dict[final_team_df.name[i]]=final_team_df.team[i]
    # removing duplicates
    df=df.drop_duplicates(subset='name',keep='first')
    df.name = df.name.astype(str)
    df.reset_index(inplace=True)
    df.drop('index',axis=1,inplace=True)
    # mapping final team in place of "TOT" representing total season stats
    for i in range(len(df)):
        if df.name.iloc[i] in list(final_team_dict.keys()):
            df.team.iloc[i] = final_team_dict[df.name.iloc[i]]
    
    return df
        

In [129]:
yrs_to_survey=[]
for i in list(range(1980,2020)):
    yrs_to_survey.append(str(i))

In [130]:
dfs=[]
for yr in yrs_to_survey:
    dfs.append(create_goalie_dataframe(yr))

In [132]:
final_df = pd.concat(dfs, ignore_index=True)

In [134]:
final_df.head()

,name,age,team,gp,gs,wins,losses,ties,goals_against,shots,...,quality_start_pct,really_bad_starts,ga_pct_minus,gs_above_avg,goals_scored,assists,points,plm,mpg,season
0,Hardy Astrom,28,CLR,49.0,,9,27,6,161,1242,...,,,110.0,-15.06,0,0,0,6,52.367,1980
1,Steve Baker,22,NYR,27.0,,9,8,6,79,667,...,,,101.0,-0.63,0,0,0,2,51.444,1980
2,Marco Baron,20,BOS,1.0,,0,0,0,2,13,...,,,0.0,0.00,0,0,0,0,40.000,1980
3,Yves Belanger,27,BOS,8.0,,2,0,3,19,132,...,,,0.0,0.00,0,0,0,0,41.000,1980
4,Dan Bouchard,29,ATF,53.0,,23,19,10,163,1598,...,,,87.0,24.77,0,1,1,34,57.830,1980


In [135]:
final_df.tail()

,name,age,team,gp,gs,wins,losses,ties,goals_against,shots,...,quality_start_pct,really_bad_starts,ga_pct_minus,gs_above_avg,goals_scored,assists,points,plm,mpg,season
3122,Cam Talbot,31,PHI,35.0,32,11,17,3,108,997,...,.281,10,120.0,-18.14,0,1,1,0,54.457,2019
3123,Linus Ullmark,25,BUF,37.0,34,15,14,5,109,1146,...,.529,6,106.0,-5.71,0,1,1,0,56.811,2019
3124,Semyon Varlamov,30,COL,49.0,49,20,19,9,136,1496,...,.510,9,101.0,-1.16,0,0,0,2,57.939,2019
3125,Andrei Vasilevskiy,24,TBL,53.0,53,39,10,4,128,1713,...,.642,7,83.0,26.40,0,3,3,6,60.453,2019
3126,Cam Ward,34,CHI,33.0,29,16,12,4,115,1113,...,.517,5,115.0,-14.68,0,1,1,4,57.030,2019


In [136]:
import dill

In [139]:
final_df.to_csv('goalie_data.csv')